In [1]:
# import libraries
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from pandas.io.json import json_normalize
import copy
import csv

# for string processing
import unidecode 
import collections
import string

# Merging names. Installation: pip install fuzzywuzzy and pip install fuzzywuzzy[speedup]
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from IPython.core import display as ICD

# for string processing
import re

%matplotlib inline 

In [2]:
def request_get_json(url):
    r = requests.get(url)
    if r.status_code != 200:
        print (r.status_code)
    page_body = r.text
    return json.loads(page_body)

In [3]:
# GO and fetch the google API keys from a local file
CENTRAL_FOLDER = '../'
API_keys = pd.read_csv(CENTRAL_FOLDER + 'API_google_key',index_col=0,header=None, sep='\=', engine='python')

#############   Google API KEYS ##################
nearest_roads_API_KEY = API_keys.iloc[0].values[0] # accepts up to 100 points
placesID_API_KEY = API_keys.iloc[1].values[0]
API_KEY = API_keys.iloc[2].values[0] # google directions

In [4]:
origin = 'Chemin+terroux' 
destination= 'Quai+du+Mont-Blanc+1201+Genève'
get_multiples_routes = 'true'


In [5]:
url = 'https://maps.googleapis.com/maps/api/directions/json?'+\
    'origin=' + origin  +\
    '&destination='+ destination  +\
    '&alternatives=' + get_multiples_routes +\
    '&key='+ API_KEY

In [6]:
my_json = request_get_json(url)
tmp = my_json['routes'][0]['legs'][0]['steps'][0]['html_instructions'][1:]
tmp

ConnectionError: HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/directions/json?origin=Chemin+terroux&destination=Quai+du+Mont-Blanc+1201+Gen%C3%A8ve&alternatives=true&key=%20AIzaSyBSd1FMoGcpWnYrOLVzOnopFq0O6foL6Es (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fc2104ba588>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [49]:

tmp1 = [m.start() for m in re.finditer('<', tmp)]
tmp2 = 

[4, 16, 24, 41, 53, 70]

In [9]:
for key, value in my_json.items() :
    print (key)

geocoded_waypoints
routes
status


In [10]:
len(my_json['routes'])

3

In [121]:
tmp0 = []
for x in my_json['geocoded_waypoints']:
    tmp0.append(x['place_id'])

In [15]:
tmp0

['ChIJWSs7wyZljEcRRb_sGE9NiBo', 'ChIJg5tQUyRljEcR1JbJsz6diro']

In [21]:
url = 'https://maps.googleapis.com/maps/api/place/details/json?'+\
            'placeid='+ tmp0[0] +'&key='+placesID_API_KEY

In [31]:
r = requests.get(url)
if r.status_code != 200:
        print (r.status_code)
page_body = r.text

In [33]:
print(r.text)

{
   "html_attributions" : [],
   "result" : {
      "address_components" : [
         {
            "long_name" : "Rue de Berne",
            "short_name" : "Rue de Berne",
            "types" : [ "route" ]
         },
         {
            "long_name" : "Genève",
            "short_name" : "Genève",
            "types" : [ "locality", "political" ]
         },
         {
            "long_name" : "Genève",
            "short_name" : "Genève",
            "types" : [ "administrative_area_level_2", "political" ]
         },
         {
            "long_name" : "Genève",
            "short_name" : "GE",
            "types" : [ "administrative_area_level_1", "political" ]
         },
         {
            "long_name" : "Switzerland",
            "short_name" : "CH",
            "types" : [ "country", "political" ]
         },
         {
            "long_name" : "1201",
            "short_name" : "1201",
            "types" : [ "postal_code" ]
         }
      ],
      "adr_address" : 

In [156]:
place_json = request_get_json(url)

In [19]:
tmp = []
start = 1997
end_ = 2001 
for i,x in enumerate(tmp0): # uncomment this line to run the ode
    url = 'https://maps.googleapis.com/maps/api/place/details/json?'+\
                'placeid='+ x +'&key='+placesID_API_KEY
    place_json = request_get_json(url) # to uncomment to do the requests
    tmp.append(place_json)
    print(i,'||', end="") # i-start to have the right index in tmp
    if tmp[i-start]['status']=='OVER_QUERY_LIMIT':
        print ('BREAK: reason is:',  tmp[i-start]['status'])
        print('the last valid index was ',i-1)
        break

AttributeError: 'dict' object has no attribute 'loads'

In [ ]:
  

# extract only the road name
tmp2 = []
for i in range(len(tmp)):
    if tmp[i]['status']=='OK':
        tmp2.append(tmp[i]['result']['name'])
    else:
        tmp2.append(':ERROR STATUS: ' + tmp[i]['status'])

# Write down roads in a csv file
df = pd.DataFrame(tmp2)
df = df[df[0] != ':ERROR STATUS: OVER_QUERY_LIMIT'] # just an additional check
df.columns = ['road_name']